In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

First, let's read in our counties shapefiles. This one was obtained from http://www.tngis.org/administrative-boundaries.htm

Shapefiles are another type of geospatial data file that you can use to create a GeoDataFrame.

In [ ]:
counties = gpd.read_file('../data/county/tncounty.shp')
counties

In [ ]:
counties['NAME']=counties['NAME'].str.upper()

Remember, if we call `.plot()` and a GeoDataFrame, it will render a plot using the geometry column.

In [ ]:
counties.plot();

If we want a larger plot, we can use `plt.subplots()` and set a figsize. When we create our plot, we need to specify that we want it to render on the axes we just created.

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
counties.plot(ax = ax);

Since the axes are not conveying useful information, we can remove them.

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
counties.plot(ax = ax)
ax.axis('off');

In [ ]:
unemploy_overdose = pd.read_csv('../data/unemploy_overdose.csv')

unemploy_overdose

In [ ]:
df1=unemploy_overdose.groupby('county').agg({'fipscounty':'min','overdose_deaths':'sum', 'unemployment_rate':'mean','state':'first','fipsstate':'max' })
df1

In [ ]:
df1.reset_index(inplace=True)
df1

In [ ]:
df1.sort_values(by='unemployment_rate',ascending=False)

In [ ]:
unemploy_overdose['overdose_deaths']=unemploy_overdose.groupby('county')['overdose_deaths'].sum()
unemploy_overdose['unemployment_rate']=unemploy_overdose.groupby('county')['unemployment_rate'].mean()
unemploy_overdose

Now, we can merge the two together. If we put the GeoDataFrame on the left, the result will be a GeoDataFrame.

In [ ]:
counties_unemp_overdose= pd.merge(left = counties, right = unemploy_overdose, 
                    left_on = 'NAME', right_on = 'county',how='inner')
counties_unemp_overdose.shape

In [ ]:
counties_unemp_overdose_df= pd.merge(left = counties, right = df1 ,
                    left_on = 'NAME', right_on = 'county',how='inner')
counties_unemp_overdose_df.shape

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))
counties_unemp_overdose_df.plot(column = 'unemployment_rate', 
              cmap = 'GnBu', 
              edgecolor = 'black', 
              legend = True,
              ax = ax)
plt.title('Unemployment Rate by County', fontsize = 14)
ax.axis('off');

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
counties_unemp_overdose_df.plot(column = 'unemployment_rate', 
              cmap = 'Greens', 
              edgecolor = 'white', 
              legend = True,
              ax = ax)
plt.title('Unemployment Rate Vs overdose_deaths by County \n   \n color : unemployment , label : overdose_deaths', fontsize = 14)
for x, y, label1,label2 in zip(counties_unemp_overdose_df.geometry.centroid.x, 
                       counties_unemp_overdose_df.geometry.centroid.y, 
                       counties_unemp_overdose_df['overdose_deaths'],
                       counties_unemp_overdose_df['county'].str.lower()):
    label = f" {label2:}\n{label1}"
    ax.annotate(str(label),xy=(x, y),horizontalalignment='center',fontsize=8,color='black',)
    ax.axis('off');
    plt.tight_layout()